In [1]:
import sys
import pandas
import numpy as np

In [3]:
def fastqReader(inFile):
    '''Reads in FASTQ files, returns a dict of header:sequence'''
    readDict = {}
    n=0
    for line in open(inFile):
        line = line.rstrip()
        n=n+1
        if not line:
            continue
        if line.startswith('@'):
            readDict[line[1:]] = ''
            lastHead = line[1:]
        elif (n%4)==2:
            readDict[lastHead] += line
    return readDict

In [4]:
R1=fastqReader(R1_fastq_file) #Miseq run with ~16 million reads

df_R1=pandas.DataFrame.from_dict(R1,orient='index')

df_R1.columns=['R1']

df_R1[['coordinate','rest']]=df_R1.index.to_series().str.split(" ", expand=True)

In [5]:
df_R1[0:5] 

,R1,coordinate,rest
M06659:228:000000000-KTJV8:1:1101:17655:1000 1:N:0:NGTTATAA+NGCGCTAG,NCACTTGAACGGGTTAACTATGTGCCGATC,M06659:228:000000000-KTJV8:1:1101:17655:1000,1:N:0:NGTTATAA+NGCGCTAG
M06659:228:000000000-KTJV8:1:1101:12267:1000 1:N:0:NGTTATAA+NGCGCTAG,NGTTGCACCCTACTACAACTCTAATGCCCA,M06659:228:000000000-KTJV8:1:1101:12267:1000,1:N:0:NGTTATAA+NGCGCTAG
M06659:228:000000000-KTJV8:1:1101:13092:1000 1:N:0:NACCGCGG+NGCGCTAG,NGTTGAGGATCCCAGACTTGTCCCGGTCTA,M06659:228:000000000-KTJV8:1:1101:13092:1000,1:N:0:NACCGCGG+NGCGCTAG
M06659:228:000000000-KTJV8:1:1101:19548:1000 1:N:0:NACCGCGG+NATATCGA,NCGCAGAACAGAAACCGTACGATCCCAGAT,M06659:228:000000000-KTJV8:1:1101:19548:1000,1:N:0:NACCGCGG+NATATCGA
M06659:228:000000000-KTJV8:1:1101:19240:1000 1:N:0:NACCGCGG+NATATCGA,NACTGTTTGCGATTTCAGTGCCCACTATCC,M06659:228:000000000-KTJV8:1:1101:19240:1000,1:N:0:NACCGCGG+NATATCGA


In [6]:
R2=fastqReader(R2_fastq_file)

df_R2=pandas.DataFrame.from_dict(R2,orient='index')

df_R2.columns=['R2']

In [7]:
len(df_R2) #this is the number of reads

16649224

In [8]:
sum(df_R2['R2']=='GATAACCCTCGGGAT') #This is a constant region expected. 
#15884550/16649224 =95.4% of R2 are as expected. Real % might be even slightly higher, because I am only counting exact matches.

15884550

In [9]:
df_R2_filtered=df_R2[df_R2['R2']=='GATAACCCTCGGGAT']

df_R2_filtered[['coordinate','rest']]=df_R2_filtered.index.to_series().str.split(" ", expand=True)

/local/50739577/ipykernel_32723/2307877900.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_R2_filtered[['coordinate','rest']]=df_R2_filtered.index.to_series().str.split(" ", expand=True)
/local/50739577/ipykernel_32723/2307877900.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_R2_filtered[['coordinate','rest']]=df_R2_filtered.index.to_series().str.split(" ", expand=True)


In [10]:
df_merge=pandas.merge(df_R2_filtered, df_R1, on='coordinate')
#Only keep those R1 reads whose R2 is with the expected sequence. by default this is an inner join

In [11]:
df_merge.set_index('R1', inplace=True)
#This is supposed to speed up merging later with the whitelist

In [13]:
#We need revcomp to use reference by Cellranger, so the annotated libC file is more useful:
libC_anno_df=pandas.read_csv(libC_construct_sequence)

/local/50739577/ipykernel_32723/2582482099.py:2: DtypeWarning: Columns (0,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  libC_anno_df=pandas.read_csv(libC_construct_sequence)


In [14]:
libC_anno_df.columns=['LineageName','SubLibrary','42ntBC','42ntBCrevcomp','LibInsertSequence']

In [15]:
libC_anno_df[0:5]

,LineageName,SubLibrary,42ntBC,42ntBCrevcomp,LibInsertSequence
0,GENBC0200001,3.0,CACTTGTTTGGTCAGGGATAACATAGGGAGTTGACGAGAAGA,TCTTCTCGTCAACTCCCTATGTTATCCCTGACCAAACAAGTG,ACCGAACGCAACGCACGCACACTTGTTTGGTCAGGGATAACATAGG...
1,GENBC0200002,3.0,CACTTGTTTGGTGACGCCTTCCCGAATCACTCCCTAATATCT,AGATATTAGGGAGTGATTCGGGAAGGCGTCACCAAACAAGTG,ACCGAACGCAACGCACGCACACTTGTTTGGTGACGCCTTCCCGAAT...
2,GENBC0200003,3.0,CACTTGTTTGGTGCGTTTCAGATCTTCCTACCTCCTTCATAC,GTATGAAGGAGGTAGGAAGATCTGAAACGCACCAAACAAGTG,ACCGAACGCAACGCACGCACACTTGTTTGGTGCGTTTCAGATCTTC...
3,GENBC0200004,3.0,CACTTGTTTGGTACCAACAGGACCTTGAAACCAACCTAATCG,CGATTAGGTTGGTTTCAAGGTCCTGTTGGTACCAAACAAGTG,ACCGAACGCAACGCACGCACACTTGTTTGGTACCAACAGGACCTTG...
4,GENBC0200005,3.0,CACTTGTTTGGTACCGAATCTACGTCTCAATGTCACCCTTGA,TCAAGGGTGACATTGAGACGTAGATTCGGTACCAAACAAGTG,ACCGAACGCAACGCACGCACACTTGTTTGGTACCGAATCTACGTCT...


In [16]:
libC_anno_df['sc_R2_30nt']=libC_anno_df['42ntBCrevcomp'].str[0:30]

In [17]:
libC_anno_df['bulk_R1_30nt']=libC_anno_df['42ntBC'].str[12:42]

In [18]:
libC_anno_df.set_index('bulk_R1_30nt', inplace=True)
#This is supposed to speed up merging

In [19]:
df_merge_valid=df_merge.merge(libC_anno_df, how='inner',left_index=True,right_index=True)
#This is now relatively fast

In [20]:
df_merge_valid.index.name="bulk_R1_30nt"
df_merge_valid.reset_index(inplace=True)

In [21]:
df_valid_count=df_merge_valid.groupby('bulk_R1_30nt').count().sort_values('coordinate', ascending=False)

In [23]:
df_valid_count['rank']=range(len(df_valid_count))

In [24]:
df_valid_count[0:5]

,R2,coordinate,rest_x,rest_y,LineageName,SubLibrary,42ntBC,42ntBCrevcomp,LibInsertSequence,sc_R2_30nt,rank
bulk_R1_30nt,,,,,,,,,,,
CAATAGTATCCGGGAATTAACACACGCCTG,3236105,3236105,3236105,3236105,3236105,3236105,3236105,3236105,3236105,3236105,0
TGTTGAGGATCCCAGACTTGTCCCGGTCTA,652037,652037,652037,652037,652037,652037,652037,652037,652037,652037,1
TTCCGGATCAAAGTACTGCGGTATCATAGG,473660,473660,473660,473660,473660,473660,473660,473660,473660,473660,2
GACTGTTTGCGATTTCAGTGCCCACTATCC,437350,437350,437350,437350,437350,437350,437350,437350,437350,437350,3
TGTTGCACCCTACTACAACTCTAATGCCCA,433253,433253,433253,433253,433253,433253,433253,433253,433253,433253,4


In [25]:
df_valid_count_cleanup=df_valid_count.reset_index()

In [26]:
df_valid_count_cleanup=df_valid_count_cleanup[['bulk_R1_30nt','R2','rank']]

In [27]:
df_valid_count_cleanup.columns=['bulk_R1_30nt','ReadCount','rank']

In [28]:
df_valid_count_cleanup[0:5]

,bulk_R1_30nt,ReadCount,rank
0,CAATAGTATCCGGGAATTAACACACGCCTG,3236105,0
1,TGTTGAGGATCCCAGACTTGTCCCGGTCTA,652037,1
2,TTCCGGATCAAAGTACTGCGGTATCATAGG,473660,2
3,GACTGTTTGCGATTTCAGTGCCCACTATCC,437350,3
4,TGTTGCACCCTACTACAACTCTAATGCCCA,433253,4


In [40]:
# df_valid_count_cleanup.to_csv('Bulk_validCount.csv',sep=',',header=True)

In [29]:
IPC298_T0_ref=libC_anno_df[libC_anno_df.index.isin(df_valid_count.index)][['LineageName','sc_R2_30nt']]

In [30]:
IPC298_T0_ref[0:5]

,LineageName,sc_R2_30nt
bulk_R1_30nt,,
CACTTTAAGGTTATGGGATGCGGGTGGTAA,GENBC0200537,TTACCACCCGCATCCCATAACCTTAAAGTG
AAGCTGAAACGCTAGGATTAAACGTCTGGG,GENBC0200739,CCCAGACGTTTAATCCTAGCGTTTCAGCTT
CAACCCGGACTACATGACTACTATCTATCC,GENBC0200896,GGATAGATAGTAGTCATGTAGTCCGGGTTG
CTGTAGTCCCGAATGTCACAGTCAGCTTAT,GENBC0200992,ATAAGCTGACTGTGACATTCGGGACTACAG
CCATCTAACACTCGCCACTCAATCGGTCGT,GENBC0201132,ACGACCGATTGAGTGGCGAGTGTTAGATGG


In [31]:
IPC298_T0_ref.columns=['id','sequence']

In [32]:
IPC298_T0_ref['name']=IPC298_T0_ref['id']
IPC298_T0_ref['read']='R2'
IPC298_T0_ref['pattern']='GACCTGCTCCAGCTGCACCA(BC)'
IPC298_T0_ref['feature_type']='CRISPR Guide Capture' #My library is for the lineage barcodes, not really guide RNA, but this is just to trick Cellranger 

IPC298_T0_ref_reorg=IPC298_T0_ref[['id','name','read','pattern','sequence','feature_type']]

In [33]:
IPC298_T0_ref_reorg[0:5]

,id,name,read,pattern,sequence,feature_type
bulk_R1_30nt,,,,,,
CACTTTAAGGTTATGGGATGCGGGTGGTAA,GENBC0200537,GENBC0200537,R2,GACCTGCTCCAGCTGCACCA(BC),TTACCACCCGCATCCCATAACCTTAAAGTG,CRISPR Guide Capture
AAGCTGAAACGCTAGGATTAAACGTCTGGG,GENBC0200739,GENBC0200739,R2,GACCTGCTCCAGCTGCACCA(BC),CCCAGACGTTTAATCCTAGCGTTTCAGCTT,CRISPR Guide Capture
CAACCCGGACTACATGACTACTATCTATCC,GENBC0200896,GENBC0200896,R2,GACCTGCTCCAGCTGCACCA(BC),GGATAGATAGTAGTCATGTAGTCCGGGTTG,CRISPR Guide Capture
CTGTAGTCCCGAATGTCACAGTCAGCTTAT,GENBC0200992,GENBC0200992,R2,GACCTGCTCCAGCTGCACCA(BC),ATAAGCTGACTGTGACATTCGGGACTACAG,CRISPR Guide Capture
CCATCTAACACTCGCCACTCAATCGGTCGT,GENBC0201132,GENBC0201132,R2,GACCTGCTCCAGCTGCACCA(BC),ACGACCGATTGAGTGGCGAGTGTTAGATGG,CRISPR Guide Capture


In [34]:
IPC298_T0_ref_reorg.shape

(643, 6)

In [57]:
IPC298_T0_ref_reorg.to_csv(feature_ref_file,sep=',',header=True,index=False,index_label=None)